In [4]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from readsql import queryToDataFrame,showTables,DescribeTable

: 

In [ ]:
showTables()

 1. Provide a visual representation of our annual sales performance over time.

In [ ]:
query = """ SELECT MONTHNAME(SaleDate) AS Month,
SUM(TotalPrice) AS total_sales
 FROM Salesdata
GROUP BY MONTH(SaleDate),MONTHNAME(SaleDate)
 ORDER BY MONTH(SaleDate),MONTHNAME(SaleDate) """

df = queryToDataFrame(query)
df
 

In [13]:
months = df['Month'].values
sales_values = df['total_sales']  = df['total_sales'].astype(float).values

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(months,sales_values,linestyle='-',linewidth=2,color='blue',marker='o',)
plt.title('Annual Sales Performance ',fontdict={'family':'Arial','size':18,'color':'darkblue'},pad=15)
plt.xlabel("months",fontdict={'family':'Arial','size':14,'color':'black'},labelpad=10)
plt.ylabel("sales_values",fontdict={'family':'Arial','size':14,'color':'black'},labelpad=10)


for i in range(len(df['Month'])):
    plt.annotate(xy=(i,df['total_sales'].values[i]),
                 text = df['total_sales'].values[i],
                 horizontalalignment='center')
    

plt.grid(linestyle = '--',color='Grey',linewidth =0.5)
plt.show()

2. The client is interested in understanding how the sales have been fluctuating
over time to identify any underlying patterns or anomalies Analyze the trend in
our monthly sales using a moving average technique.

In [ ]:
query = """select
months,
AVG(monthlysales) OVER (ORDER BY monthnum ROWS BETWEEN 3
PRECEDING AND CURRENT ROW) AS MovingAverage
from
(SELECT
month(saledate) monthnum,
monthname(SaleDate) as months,
sum(totalprice) as monthlysales
FROM Salesdata
Group by month(saledate),monthname(saledate)) t;"""

df = queryToDataFrame(query)
df

In [35]:
months = df['months'].values
Moving_Average = df['MovingAverage'] = df['MovingAverage'].astype(float).values

In [ ]:
df.info()


In [ ]:
plt.figure(figsize=(12,6))

plt.title('Moving Average Over Time',  fontdict={'family':'Arial','size':20,'color':'darkblue'},pad=20)
plt.plot(months,Moving_Average,marker='o',color='green')
plt.xlabel('months',fontdict={'family':'serif','size':14,'color':'black'},labelpad=10)
plt.ylabel('MovingAverage',fontdict={'family':'Arial','size':14,'color':'black'},labelpad=10)


for i in range(len(df['months'])):
    plt.annotate(xy=(i, df['MovingAverage'].values[i]), 
                 text=df['MovingAverage'].values[i], 
                 horizontalalignment='center')


plt.grid(linestyle = '--',color='Grey',linewidth =0.5)
plt.show()


 3. Explore the month-over-month growth rate in sales.

In [ ]:
query = """ select
 months,
 100*(monthlysales-lag(monthlysales) over(order by monthnum))/
 lag(monthlysales) over(order by monthnum) as 'growthrate in %'
 from
 (SELECT
 month(saledate) monthnum,
 monthname(SaleDate) as months,
 sum(totalprice) as monthlysales
 FROM Salesdata
 Group by month(saledate),monthname(saledate)) t
 order by monthnum,months;"""

df = queryToDataFrame(query)
df

In [54]:
Months = df['months'].values
growthrate=df['growthrate in %']= df['growthrate in %'].astype('float').values

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Growth Rate Over Time', fontdict={'family':'Arial','size':20,'color':'darkblue'},pad=20)
plt.plot(Months, growthrate, marker='o', color='teal')

plt.xlabel('Month', fontdict={'family':'Arial','size':17,'color':'black'},labelpad=15)
plt.ylabel('Growth Rate (%)', fontdict={'family':'Arial','size':17,'color':'black'},labelpad=15)

for i in range(len(Months)):
    plt.annotate(xy=(Months[i], growthrate[i]),
                 text=growthrate[i],
                 horizontalalignment='center')

plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()


4. identify the top-performing sales representatives based on their total sales
volume

In [ ]:
query = """select t.employeeid,e.firstname,t.totalsales
 from
 (select employeeid,sum(totalprice) as totalsales
 from salesdata
 group by EmployeeID) t
 join employees e
 on t.employeeid=e.EmployeeID
 order by t.totalsales desc
 limit 10;"""


df = queryToDataFrame(query)
df

In [69]:
employees = df ['firstname'].values
Totalsales = df ['totalsales']= df['totalsales'].astype(float).values

In [ ]:
df.info()

In [72]:
cmap_val = np.linspace(0.3,0.7,10)
cmap = plt.cm.Blues(cmap_val)

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(employees,Totalsales,color=cmap,edgecolor='black')
plt.plot(employees,Totalsales, marker='o', color= 'Darkblue',linewidth=2)
plt.title('Employee Sales Performance', fontdict={'family':'Arial','size':20,'color':'black'},pad=20)

plt.xlabel('Month', fontdict={'family':'Arial','size':17,'color':'Black'},labelpad=15) 
plt.ylabel('Growth Rate (%)',fontdict={'family':'Arial','size':17,'color':'black'},labelpad=15)

for i in range(len(Totalsales)):
    plt.annotate(xy=(employees[i], Totalsales[i]),
                 text=Totalsales[i],
                 horizontalalignment='center')


plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()

5. what are the total sales made by each employee?

In [ ]:
query = """select firstname,sum(totalprice) as total
from employees e
join salesdata s
on e.employeeid = s.employeeid
group by firstname
order by total desc
limit 10;"""

df = queryToDataFrame(query)
df

In [88]:
employees = df['firstname'].values
Totalsales = df['total'] = df['total'].astype('float').values

In [ ]:
df.info()

In [97]:
cmap_val = np.linspace(0.3,0.7,10)
cmap = plt.cm.Accent(cmap_val)

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(employees,Totalsales ,color=cmap,edgecolor='black')
plt.title('Employee Sales Performance', fontdict={'family':'Arial','size':20,'color':'black'},pad=20)
plt.xlabel('employees', fontdict={'family':'Arial','size':17,'color':'Black'},labelpad=15) 
plt.ylabel('Tolal_sales',fontdict={'family':'Arial','size':17,'color':'black'},labelpad=15)


plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()

6. Which category has the highest total sales ?

In [ ]:
query = """ SELECT
 c.CategoryName,
 SUM(s.TotalPrice) AS TotalSales
 FROM Salesdata s
 JOIN Subcategories sc ON s.SubcatID = sc.SubcatID
 JOIN Categories c ON sc.CategoryID = c.CategoryID
 GROUP BY c.CategoryName
 ORDER BY TotalSales DESC;"""

df = queryToDataFrame(query)
df

In [ ]:
Category = df['CategoryName'].values
Total_sales = df['TotalSales'] = df['TotalSales'].astype('float').values

df.info()

In [ ]:
cmap_val = np.linspace(0.3,0.7,10)
cmap = plt.cm.PuBu(cmap_val)

In [ ]:
plt.figure(figsize=(12, 6))
plt.pie(x=Total_sales,labels=Category,autopct='%.2f%%',
        explode=(0.1,0,0,0,0,0),
       labeldistance=1.3,
       colors=cmap)
   
plt.show()

7. How many customers are there in each state?

In [ ]:
query = """SELECT
State,
COUNT(*) AS TotalCustomers
FROM Customers
GROUP BY State
ORDER BY TotalCustomers desc
limit 10;"""

df = queryToDataFrame(query)
df

In [ ]:
states = df['State'].values
No_of_customers = df['TotalCustomers'].values
df.info()

In [ ]:
cmap_val = np.linspace(0.2,0.5,10)
cmap = plt.cm.viridis(cmap_val)

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("No of Customers by States",fontdict={'family':'Arial','size':20,'color':'black'},pad=20 )
plt.bar(states,No_of_customers, color=cmap)

plt.xlabel('States',fontdict={'family':'Arial','size':15,'color':'black'},labelpad=15)
plt.ylabel('No_of_customers', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=15)

for i in range(len(states)):
    plt.annotate(xy=(i, No_of_customers[i]),
                 text=str(No_of_customers[i]),
                 horizontalalignment='center',size=10)
            

plt.xticks(rotation=90,fontsize=12)
plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()

8. What is the average quantity of each product sold ? 

In [ ]:
query = """ SELECT s.SubcatID,sb.SubcatName ,AVG(Quantity) AS AvgQuantitySold
 FROM Salesdata s
 join subcategories sb on s.subcatid=sb.SubcatID
 GROUP BY s.SubcatID,sb.subcatname;"""

df = queryToDataFrame(query)
df

In [ ]:
Categories = df['SubcatName'].values
quantity = df['AvgQuantitySold'] = df['AvgQuantitySold'].astype('float').values

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(Categories,quantity ,color=cmap,edgecolor='black')
plt.title('Avg_quantity_sold', fontdict={'family':'Arial','size':20,'color':'black'},pad=20)
plt.xlabel('Quantity Sold', fontdict={'family':'Arial','size':17,'color':'Black'},labelpad=15) 
plt.ylabel('SubcatName',fontdict={'family':'Arial','size':17,'color':'black'},labelpad=15)


plt.xticks(rotation=90)
plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()

9. Which supplier has the highest total sales?

In [ ]:
query = """SELECT
SupplierName,
SUM(TotalPrice) AS TotalSales
FROM Salesdata s
JOIN Subcategories sc ON s.SubcatID = sc.SubcatID
JOIN Suppliers su ON sc.SupplierID = su.SupplierID
GROUP BY SupplierName
ORDER BY TotalSales DESC
limit 10;"""

df = queryToDataFrame(query)
df

In [ ]:
Suppliername = df['SupplierName'].values
Totalprice = df['TotalSales'] = df['TotalSales'].astype(float).values
df.info()

In [ ]:
cmap_val = np.linspace(0.3,0.7,10)
cmap = plt.cm.magma(cmap_val)

In [ ]:
plt.figure(figsize=(14,7))
plt.title(' Sales by supplier', fontdict={'family':'Arial','size':20,'color':'black'},pad=20 )
plt.bar(Suppliername, Totalprice, color=cmap)

plt.xlabel('Category', fontdict={'family':'Arial','size':20,'color':'black'},labelpad=10)
plt.ylabel('Sales', fontdict={'family':'Arial','size':20,'color':'black'},labelpad=15)

for i in range(len(Suppliername)):
    plt.annotate(xy=(i, Totalprice[i]),
                 text=str(Totalprice[i]),
                 horizontalalignment='center',)

plt.xticks(rotation=90)
plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()


10. What is the total revenue generated from sales in each state?

In [ ]:
query = """SELECT
c.State,
SUM(s.TotalPrice) AS TotalRevenue
FROM Salesdata s
JOIN Customers c ON s.CustomerID = c.CustomerID
GROUP BY c.State
limit 10;"""

df = queryToDataFrame(query)
df

In [ ]:
State = df['State'].values
Total_Revenue = df['TotalRevenue'] = df['TotalRevenue'].astype(float).values
df.info()

In [ ]:
cmap_val = np.linspace(0.4,0.9,10)
cmap = plt.cm.Blues(cmap_val)

In [ ]:
plt.figure(figsize=(14, 7))
plt.title('State-wise Total Sales Revenue', fontdict={'family':'Arial','size':20,'color':'black'},pad=20)
plt.bar(State, Total_Revenue, color=cmap)

plt.xlabel('States', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)
plt.ylabel('Sales', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)

for i in range(len(State)):
    plt.annotate(xy=(i, Total_Revenue[i]),
                 text=str(Total_Revenue[i]),
                 horizontalalignment='center')

plt.xticks(rotation=45)
plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()

11. How many sales were made by each customer. ?

In [ ]:
query = """SELECT
s.CustomerID,
c.firstname,
COUNT(*) AS TotalSales
FROM Salesdata s
join customers c
on s.CustomerID=c.CustomerID
GROUP BY s.CustomerID,c.firstname
order by TotalSales desc
limit 10;"""

df = queryToDataFrame(query)
df

In [ ]:
Customer_Names = df['firstname'].values
Total_sales = df['TotalSales'] = df['TotalSales'].astype(float).values
df.info()

In [ ]:
cmap_val = np.linspace(0.4,0.8,10)
cmap = plt.cm.YlGn(cmap_val)

In [ ]:
plt.figure(figsize=(14, 7))
plt.title('Top Customers With Highest Sales', fontdict={'family':'Arial','size':20,'color':'black'},pad=20)
plt.bar(Customer_Names, Total_sales, color=cmap)

plt.xlabel('Customer_Names', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)
plt.ylabel('Total Sales', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)

for i in range(len(Customer_Names)):
    plt.annotate(xy=(i, Total_sales[i]),
                 text=str(Total_sales[i]),
                 horizontalalignment='center') 

plt.xticks(rotation=45)
plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()


12. What is the total salary expenditure on employees in each department?

In [ ]:
query = """SELECT
Department,
SUM(Salary) AS TotalSalaryExpenditure
FROM Employees
GROUP BY Department;"""

df = queryToDataFrame(query)
df

In [ ]:
Departments = df['Department'].values
Expenditure = df['TotalSalaryExpenditure'] = df['TotalSalaryExpenditure'].astype(float).values
df.info()

In [ ]:
cmap_val = np.linspace(0.3,1,10)
cmap = plt.cm.BuGn(cmap_val)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Departmen Expenditure', fontdict={'family':'Arial','size':20,'color':'black'},pad=10)
plt.bar(Departments, Expenditure, color=cmap)

plt.xlabel('Departments', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)
plt.ylabel('Salary',  fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)

for i in range(len(Departments)):
    plt.annotate(xy=(i, Expenditure[i]),
                 text=str(Expenditure[i]),
                 horizontalalignment='center')

# plt.xticks(rotation=45)
plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()


# 13.Which subcategory has the highest sellinge price ?

In [ ]:
query = """SELECT
 SubcatName,
 AVG(UnitPrice) AS AvgSellingPrice
 FROM Subcategories
 GROUP BY SubcatName
 ORDER BY AvgSellingPrice DESC
 limit 5;"""

df = queryToDataFrame(query)
df

In [ ]:
subcategory = df['SubcatName'].values
selling_price = df['AvgSellingPrice'] = df['AvgSellingPrice'].astype('float').values

In [ ]:
df.info()

In [ ]:
cmap_val = np.linspace(0.3,1,10)
cmap = plt.cm.BuPu(cmap_val)

In [ ]:
plt.figure(figsize=(12, 6))
plt.pie(x=selling_price,labels=subcategory,autopct='%.2f%%',
        explode=(0.1,0,0,0,0),
       labeldistance=1.3,
       colors=cmap)
   
plt.show()

14. What is the total revenue generated from sales in each category

In [ ]:
query = """SELECT
 c.CategoryName,
 SUM(s.TotalPrice) AS TotalRevenue
 FROM Salesdata s
 JOIN Subcategories sc ON s.SubcatID = sc.SubcatID
 JOIN Categories c ON sc.CategoryID = c.CategoryID
 GROUP BY c.CategoryName;"""

df = queryToDataFrame(query)

In [ ]:
Categoryname = df['CategoryName'].values
Totalsales = df['TotalRevenue'] = df['TotalRevenue'].astype(float).values
df.info()


In [ ]:
cmap_val = np.linspace(0.2,1,10)
cmap = plt.cm.BuPu(cmap_val)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Revenue From Each Categories', fontdict={'family':'Arial','size':20,'color':'black'},pad=10)
plt.bar(Categoryname, Totalsales, color=cmap)

plt.xlabel('Categories', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)
plt.ylabel('Sales', fontdict={'family':'Arial','size':15,'color':'black'},labelpad=10)

for i in range(len(Categoryname)):
    plt.annotate(xy=(i, Totalsales[i]),
                 text=str(Totalsales[i]),
                 horizontalalignment='center')

plt.xticks(rotation=90)
plt.grid(axis='y',linestyle='--',color='Grey',linewidth=0.5)
plt.tight_layout()
plt.show()
